In [5]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import pandas as pd
import sqlalchemy as sa

import database as db

from images import copy_images, get_list_of_images_invalid, remove_images_invalid, save_metadata
from tables import County, DataTrustedIdentifier, InfoImage

## Database connection

In [7]:
engine, session = db.connect()
engine.echo = False

2023-03-23 22:43:52,596 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-23 22:43:52,597 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-23 22:43:52,598 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-23 22:43:52,599 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-23 22:43:52,600 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-23 22:43:52,600 INFO sqlalchemy.engine.Engine [raw sql] {}


## List of images invalid

In [10]:
list_images_invalid = get_list_of_images_invalid()
pd.DataFrame(list_images_invalid).head(3)

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal


## BR dataset

In [8]:
query = session.query(County).distinct().all()

uf_unaccented_lower, state_unaccented_lower, county_unaccented_lower = db.get_list_uf_state_county(query)

## Define variable for build dataset

In [9]:
level = DataTrustedIdentifier.specific_epithet
columns = [level, sa.func.array_agg(DataTrustedIdentifier.seq)]
condition = sa.and_(DataTrustedIdentifier.country_trusted =='Brasil',
                DataTrustedIdentifier.specific_epithet.is_not(None),
                 sa.or_(uf_unaccented_lower, state_unaccented_lower))
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'

## copy images

In [11]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            records = db.get_records_group_by_level(columns, condition, level, minimum_image, session)
            list_level_name, list_path_images = db.filter_records(color, image_size, minimum_image, records, session)

            if len(list_path_images) > 0:
                list_count_path, list_path_images = remove_images_invalid(list_level_name, list_images_invalid, list_path_images)

                path_out = os.path.join(path_out, color.upper(), image_size, str(minimum_image))

                if not os.path.exists(path_out):
                    os.makedirs(path_out)

                list_path_dst = copy_images(list_level_name, list_path_images, path_out)

                save_metadata(color, image_size, list_level_name, list_path_images, list_count_path, list_path_dst, minimum_image, path_out, session)

/home/xandao/Documentos/piperaceae-sql/dataset_br/database.py:47: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  records = session.query(*columns) \


specie: aleyreanum before: 54 after: 53
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0023115.jpeg']
specie: arboreum before: 621 after: 620
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0248528.jpeg']
specie: bartlingianum before: 146 after: 145
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421575_01.jpeg']
specie: belterraense before: 14 after: 13
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/NL-U1484137.jpeg']
specie: blanda before: 114 after: 113
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/HUFSJ001133_v00.jpeg']
specie: callosum before: 60 after: 59
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0022379_nd.jpeg']
specie: decora before: 39 after: 38
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/HUFSJ002198_v00.jpeg']
specie: hispidum before: 332 after: 331
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0032742_nd.jpeg']
specie: hostmannianum before: 146 after: 144
diff: ['/home/xandao/Documentos/RGB/512/w_pred_mask/

'color: rgb image_size: 512 minimum_image: 5'

'len(list_level_name): 235 len(paths_images): 10512'

'save csv in /home/xandao/Imagens/br_dataset/RGB/512/5/dataset_informations.csv'

...

,species,paths,count,dst
0,aduncum,[/home/xandao/Documentos/RGB/512/w_pred_mask/A...,377,/home/xandao/Imagens/br_dataset/RGB/512/5/RGB/...
1,aequale,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,30,/home/xandao/Imagens/br_dataset/RGB/512/5/RGB/...
2,alata,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,81,/home/xandao/Imagens/br_dataset/RGB/512/5/RGB/...


'total of images 9235'

'image informations'

'save csv in /home/xandao/Imagens/br_dataset/RGB/512/5/RGB/512/10/RGB/512/20/image_informations.csv'

,seq,genus,specific_epithet,catalog_number,barcode,path_image,institution_code,collection_code,url
0,12548,Piper,dilatatum,54499,CEN00054499,/home/xandao/Documentos/RGB/512/w_pred_mask/CE...,Cenargen,CEN,https://specieslink.net/search/util/osd-dezoom...
1,11290,Peperomia,elongata,30640,CEN00030640,/home/xandao/Documentos/RGB/512/w_pred_mask/CE...,Cenargen,CEN,https://specieslink.net/search/util/osd-dezoom...
2,11292,Peperomia,glabella,45015,CEN00045015,/home/xandao/Documentos/RGB/512/w_pred_mask/CE...,Cenargen,CEN,https://specieslink.net/search/util/osd-dezoom...


## Close connection

In [ ]:
session.close()
engine.close()